In [ ]:
import pandas as pd
from rdkit import Chem

In [ ]:
path =r"Q:\ALK_1_Wild\ALK_300nM_External\ChEMBL_TargetID_CHEMBL4247_1810_External.csv"# csv文件路径
smi_columnName = 'Smiles'   # Smiles列的列名
other_columnNames = ['index']  

In [ ]:
def SmilesToSDF(path,smi_columnName='Smiles',other_columnNames=None):
    """将Smiles转成sdf文件
        参数：
        -----
        path：csv文件路径
        smi_columnName：Smiles列的列名
        other_columnNames：其他要保存在sdf文件的信息的列名(一个或多个都行)

        例子：
        -----
        path = "C:/OneDrive/Scholar/Data/sPLA2_558_conv.csv"
        smi_columnName = 'SMILES'
        other_columnNames = ['Compound_ID','Activity']
        SmilesToSDF(path,smi_columnName=smi_columnName,other_columnNames=other_columnNames)"""
    data = pd.read_csv(path)
    mols = []
    new_smi = []
    for i,smi in enumerate(data.loc[:,smi_columnName]):
        m = Chem.MolFromSmiles(smi)
        if m is not None:
            m.SetProp('_Smiles',smi)
            m.SetIntProp('init_index',i+1)
            csmi = Chem.MolToSmiles(m)
            m.SetProp('RDKit_Canonical_SMILES',csmi)
            new_smi.append(csmi)
            if other_columnNames is not None:
                if type(other_columnNames) == str:
                    other_columnNames = [other_columnNames]
                for column in other_columnNames:
                    column_value = data.loc[:,column][i]
                    m.SetProp(column,str(column_value))

            mols.append(m)
        else:
            print('The {}th smiles is invalid!!'.format(i+1))
    out_path = path[:path.rfind('.')]+'.sdf'
    wt = Chem.SDWriter(out_path)
    for m in mols:
        wt.write(m)
    wt.close()
    print('The SD file have been saved in \033[1m{}\033[0m'.format(out_path))

SmilesToSDF(path,smi_columnName=smi_columnName,other_columnNames=other_columnNames)
